In [ ]:
from dateutil.parser import parse 
import matplotlib as mpl
from matplotlib.pylab import rcParams

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
#Read data
VerkehrsZaehlung = pd.read_csv("/Users/HF/Library/Mobile Documents/com~apple~CloudDocs/Velo-Zueri-Unfaelle/data/2021_verkehrszaehlungen_werte_fussgaenger_velo.csv")
#VeloWege = gpd.read_file(r'data/Velo_wege.gpkg')
#Unfaelle = gpd.read_file(r'data/RoadTrafficAccidentLocations.gpkg')

# Turn it into a GeoDataFrame with point geometries
Zaehlungen = gpd.GeoDataFrame(
    VerkehrsZaehlung, geometry=gpd.points_from_xy(VerkehrsZaehlung.OST, VerkehrsZaehlung.NORD))

In [ ]:
Zaehlungen = Zaehlungen.drop(["FUSS_OUT", "FUSS_IN"], axis = 1)
Zaehlungen.head()

In [ ]:
# Aggregiert Daten auf eine Zählung pro Stunde. Daten von einzelner Zählstation 
# werden noch nicht wieder zusammengefügt. Koordinaten können nicht behalten werden,
# können aber über den Standort wieder eingefügt werden.
def aggregation(inp):
    list_stations = inp.FK_STANDORT.unique()
    aggregated_list = []

    for standort in list_stations:
        station = inp.loc[inp["FK_STANDORT"] == standort]
        coordinates = inp.loc[inp["FK_STANDORT"] == standort, ["FK_STANDORT", "geometry"]]
        station["Date_Hour"] = station["DATUM"].astype(str).str[:13]
        grouped = station.groupby("Date_Hour").agg({"FK_STANDORT": min, "VELO_IN": sum, "VELO_OUT": sum})
        
        #merged = pd.merge(grouped, coordinates)
        #merged = pd.concat([grouped, coordinates], axis = 1)
        merged = pd.merge(grouped, coordinates, on="FK_STANDORT")
        aggregated_list.append(merged)
    
    aggregated = pd.concat(aggregated_list)

    return aggregated

test = aggregation(Zaehlungen)